In [1]:
import requests
from bs4 import BeautifulSoup
import re
import json

In [2]:
volume = 30
issue = 1
kv = {'user-agent': 'Mozilla/5.0'}
url = "https://academic.oup.com/rfs/issue/{:}/{:}".format(volume, issue)
print(url)

https://academic.oup.com/rfs/issue/30/1


In [3]:
r = requests.get(url, headers=kv, timeout=30)
r.raise_for_status()
r.status_code
r.encoding = r.apparent_encoding

In [4]:
r.request.headers

{'user-agent': 'Mozilla/5.0', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'Connection': 'keep-alive'}

In [5]:
r.text[:1000]

'\r\n\r\n<!DOCTYPE html>\r\n<html lang="en">\r\n<head><meta name="viewport" content="width=device-width, initial-scale=1, maximum-scale=1" /><meta charset="utf-8" /><meta http-equiv="Content-Type" content="text/html; charset=utf-8" /><meta http-equiv="X-UA-Compatible" content="IE=Edge" /><!-- Bookmark Icons -->\r\n  <link rel="apple-touch-icon" sizes="180x180" href="//oup.silverchair-cdn.com/UI/app/img/v-637502067956838175/apple-touch-icon.png">\r\n  <link rel="icon" type="image/png" href="//oup.silverchair-cdn.com/UI/app/img/v-637502067956888256/favicon-32x32.png" sizes="32x32">\r\n  <link rel="icon" type="image/png" href="//oup.silverchair-cdn.com/UI/app/img/v-637502067956888256/favicon-16x16.png" sizes="16x16">\r\n  <link rel="mask-icon" href="//oup.silverchair-cdn.com/UI/app/img/v-637502067957088168/safari-pinned-tab.svg" color="#5bbad5">\r\n  <link rel="shortcut icon" href="//oup.silverchair-cdn.com/UI/app/img/v-637502067956888256/favicon.ico">\r\n  <link rel="manifest" href="//ou

In [6]:
soup = BeautifulSoup(r.text, 'html.parser')


## get article links

In [7]:
article_links = []

for link in soup.findAll('a'):
    try: 
        href = link.get('href')
        # print(href)
        search = re.search(r'/rfs/article/\d{1,2}/\d{1,2}/\d{1,3}/\d{7}', href)
        if search:
            # print("\n found article link")
            # print(search.group(0))
            # print('\n')
            article_links.append(search.group(0))
    except:
        pass


article_links = list(set(article_links))
article_links

['/rfs/article/30/1/120/2669961',
 '/rfs/article/30/1/281/2669958',
 '/rfs/article/30/1/162/2439511',
 '/rfs/article/30/1/2/2682977',
 '/rfs/article/30/1/80/2669972',
 '/rfs/article/30/1/211/2528255',
 '/rfs/article/30/1/245/2669940',
 '/rfs/article/30/1/324/2669974',
 '/rfs/article/30/1/48/2669965',
 '/rfs/article/30/1/1/2732656']

In [8]:
link = "/rfs/article/34/1/149/5781971#supplementary-data"

In [9]:
re.search(r'/rfs/article/\d{1,2}/\d{1,2}/\d{1,3}/\d{7}', link)

<re.Match object; span=(0, 29), match='/rfs/article/34/1/149/5781971'>

In [10]:
re.search(r'/rfs/article/\d{1,2}/\d{1,2}/\d{1,4}/\d{7}', link).group(0)

'/rfs/article/34/1/149/5781971'

## get free articles id

In [11]:
soup.find('div', 'widget widget-ArticleListAccess widget-instance-OUP_ArticleListAccess').find('script')

<script type="text/javascript">

        /*******************************************************************************
         * JS here is only being used to assign variables from values in the model
         * logic should be implemented in external JS files, like client.script.js
         *******************************************************************************/

        var SCM = SCM || {};
        var accessIcons = [{"id":2669961,"icon":"icon-availability_cart","title":"Available for Purchase"},{"id":2528255,"icon":"icon-availability_cart","title":"Available for Purchase"},{"id":2669972,"icon":"icon-availability_free","title":"Free"},{"id":2669974,"icon":"icon-availability_cart","title":"Available for Purchase"},{"id":2682977,"icon":"icon-availability_free","title":"Free"},{"id":2732656,"icon":"icon-availability_free","title":"Free"},{"id":2439511,"icon":"icon-availability_cart","title":"Available for Purchase"},{"id":2669940,"icon":"icon-availability_cart","title":"Avai

In [12]:
import json
free_article_id = []
script = soup.find('div', 'widget widget-ArticleListAccess widget-instance-OUP_ArticleListAccess').find('script').contents[0]
free_articles = re.findall('{"id":\d{7},"icon":"icon-availability_free","title":"Free"}', script)
for free_article in free_articles:
    # print(free_article)
    parsed = json.loads(free_article)
    id = parsed['id']
    free_article_id.append(id)

free_article_id

[2669972, 2682977, 2732656, 2669965]

## get free article links

In [13]:
free_article_links = []
for link in article_links:
    if int(link.split('/')[-1]) in free_article_id:
        # print(link)
        free_article_links.append(link)

free_article_links

['/rfs/article/30/1/2/2682977',
 '/rfs/article/30/1/80/2669972',
 '/rfs/article/30/1/48/2669965',
 '/rfs/article/30/1/1/2732656']